In [2]:
import pandas as pd
import numpy as np

## Portfolio

- SPY: €300 000
- AEX: €300 000
- RDSA: €10 000
- HSI: €90 000
- Bonds: €300 000

Total: €1 000 000

In [3]:
data = pd.read_csv('dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [7]:
relative_weights = [0.3, 0.3, 0.01, 0.09, 0.3]
absolute_weights = [300000, 300000, 10000, 90000, 300000]

In [20]:
stock_returns = np.log(data[['SPY', 'AEX', 'RDSA.AS', 'HSI']]).diff()
stock_returns[['SPY', 'HSI']] = stock_returns[['SPY', 'HSI']].values + np.log(data[['USDEUR', 'HKDEUR']]).diff().values
yields = data['YIELDS'] / 100

### Real losses

In [21]:
losses = np.exp(stock_returns) - 1
until_maturity = pd.Series(np.arange(12, 0, -1/(250))[:len(data)])
until_maturity.index = data.index
losses['bonds'] = np.exp(-until_maturity * yields.diff() + (yields + yields.diff()) / 250) - 1
losses = -losses

In [22]:
total_losses = (losses * relative_weights).sum(axis=1)
total_losses

Date
2010-01-01    0.000000
2010-01-04    0.000552
2010-01-05   -0.004411
2010-01-06    0.001525
2010-01-07   -0.003465
                ...   
2020-04-20    0.000435
2020-04-21    0.021232
2020-04-22   -0.017767
2020-04-23   -0.002725
2020-04-24    0.004614
Length: 2992, dtype: float64

### Linear losses

In [23]:
linear_losses = stock_returns
linear_losses['bonds'] = yields / 250 - until_maturity * yields.diff()
linear_losses = -linear_losses

In [24]:
total_linear_losses = (linear_losses * relative_weights).sum(axis=1)
total_linear_losses

Date
2010-01-01    0.000000
2010-01-04    0.000553
2010-01-05   -0.004374
2010-01-06    0.001528
2010-01-07   -0.003446
                ...   
2020-04-20    0.000443
2020-04-21    0.021567
2020-04-22   -0.017525
2020-04-23   -0.002713
2020-04-24    0.004657
Length: 2992, dtype: float64

## Linear loss as a function of risk factors

In [25]:
z = np.log(data[['SPY', 'AEX', 'RDSA.AS', 'HSI', 'USDEUR', 'HKDEUR']])
z['YIELDS'] = data['YIELDS'] / 100
x = z.diff()

c = np.zeros(x.shape)
c[:,-1] = z['YIELDS'] / 250 * 0.3

b = np.zeros(x.shape)
b[:,:] = [0.3, 0.3, 0.01, 0.09, 0.3, 0.09, 0.3]
b[:,-1] *= -np.arange(12, 0, -1/(250))[:len(x)]

l = -(c + b * x)

l

,SPY,AEX,RDSA.AS,HSI,USDEUR,HKDEUR,YIELDS
Date,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,-0.000000,-0.000000,-0.000000,-0.000000,0.000709,-0.000111,-0.000045
2010-01-05,-0.000793,0.000885,-0.000067,-0.001862,-0.001209,-0.000372,-0.000955
2010-01-06,-0.000211,0.000474,0.000072,-0.000552,0.000794,0.000211,0.000741
2010-01-07,-0.001264,0.000818,0.000030,0.000593,-0.001797,-0.000501,-0.001325
...,...,...,...,...,...,...,...
2020-04-20,0.001788,-0.001357,0.000027,0.000062,-0.000059,-0.000024,0.000005
2020-04-21,0.009250,0.010115,0.000488,0.002007,-0.000231,-0.000062,0.000001
2020-04-22,-0.006586,-0.008272,-0.000603,-0.000377,-0.001297,-0.000400,0.000009
